In [ ]:
from pathlib import Path
from importlib import reload
from IPython.display import display
import json

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
folder_name = 'bu'
pattern = '*.mp4'

In [ ]:
curr_dir = Path('./').resolve()
display(curr_dir)

data_dir = Path('/mnt/nfs/buses')
display(data_dir)

videos_dir = data_dir/folder_name
display(videos_dir)
print('{} videos in videos_dir'.format(len(list(videos_dir.glob(pattern)))))

out_dir = data_dir/'{}_res_deepsort'.format(folder_name)
out_dir.mkdir(exist_ok=True)
display(out_dir)

In [ ]:
from PIL import Image
import tensorflow as tf

from detector import detector

from tools import generate_detections as gdet
from deep_sort import nn_matching
from deep_sort.tracker import Tracker
from utils import read_frames
import demo

In [ ]:
sess = tf.Session()
yolo = detector.YOLOv3(
    session=sess,
    gpu_num=0,
    class_names='/mnt/nfs/detectors/yolov3-tf/names/persons.names', 
    weights_file='/mnt/nfs/yolov3-persons_final0609.weights',
    size=640,
)

In [ ]:
# Definition of the parameters
max_cosine_distance = 0.3
nn_budget = None

# deep_sort 
model_filename = 'model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)

metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

In [ ]:
def plot_image(
    image,
    title: str='',
    *,
    figsize: tuple=(20, 5),
    boxes: list=[],
    extra_operations=lambda: None
):
    '''
    boxes - list of tuples of (x_min, x_max, y_min, y_max)
        remember that matplotlib's coordinates x is horizontal, y is vertical
    extra_operations - lambda with everything you want to do to plt
    '''
    plt.figure(figsize=figsize)
    plt.imshow(
        image,
#         origin='lower', # for axis to be right pair of verctors
    )
    plt.title(title)
    plt.xlabel('y')
    plt.ylabel('x')
    for box in boxes:
        rect = patches.Rectangle(
            (box[0], box[1]),
            box[2],
            box[3],
            linewidth=3,
            edgecolor='g',
            facecolor='none',
        )
        plt.gca().add_patch(rect)
    extra_operations()
    plt.show()

In [ ]:
video_paths = [
    videos_dir/name for name in [
        '2018-12-17_17-50-34.mp4',
        '2018-12-17_15-48-54.mp4',
        '2018-12-17_16-16-57.mp4',
        '2018-12-16_18-03-55.mp4',
        '2018-12-16_17-36-55.mp4',
    ]
]

In [ ]:
# check yolo detections on one frame or video
for i, video_path in enumerate(video_paths):
    for frame in read_frames(video_path):
        image = Image.fromarray(frame)
        yolo_dets = yolo.predict_tracker(image)
        boxes = [[int(d) for d in det[2:6]] for det in yolo_dets]
        print('boxes', boxes)
        plot_image(
            frame,
            figsize=(20, 10),
            boxes=boxes,
        )
        break
    break

In [ ]:
for video_path in out_dir.glob(pattern):
    print(video_path)
    yolo_dets_path = video_path.with_suffix('.yolo.json')
    with open(yolo_dets_path) as file:
        yolo_dets = json.load(file)
    print(len(yolo_dets))
    display(yolo_dets[0])
    
    for frame, dets in zip(read_frames(video_path), yolo_dets):
        plot_image(
            frame,
            figsize=(20, 10),
            boxes=boxes,
        )
        break

In [ ]:
a = np.arange(10)
print(a)

In [ ]:
%%time

reload(demo)

# reload_tracker(tracker)

reports = []

# for i, video_path in enumerate(videos_dir.glob(pattern)):
for i, video_path in enumerate(video_paths):
    print('processing video:', i, video_path)
    report = demo.process_video(video_path, out_dir, yolo, metric, tracker, encoder, write_video=True, write_yolo=True)
    reports.append(report)
    tracker.reset()

    if i > -1: break

with open(out_dir/'report_{}.json'.format(folder_name), 'w') as file:
    json.dump(reports, file)